# Test Data

In [23]:
import os
import pandas as pd

prefix = '<path_to>/fov_segmentation'

df = pd.read_pickle(os.path.join(prefix, 'light_log.pkl'))
test_df = df[df.test == True]
test_df

,image,segmentation,test
8,"{'folder': 'frame', 'file': '0008.npy'}","{'folder': 'segmentation', 'file': '0008.npy'}",True
9,"{'folder': 'frame', 'file': '0009.npy'}","{'folder': 'segmentation', 'file': '0009.npy'}",True
10,"{'folder': 'frame', 'file': '0010.npy'}","{'folder': 'segmentation', 'file': '0010.npy'}",True
12,"{'folder': 'frame', 'file': '0012.npy'}","{'folder': 'segmentation', 'file': '0012.npy'}",True
17,"{'folder': 'frame', 'file': '0017.npy'}","{'folder': 'segmentation', 'file': '0017.npy'}",True
...,...,...,...
1482,"{'folder': 'frame', 'file': '1482.npy'}","{'folder': 'segmentation', 'file': '1482.npy'}",True
1484,"{'folder': 'frame', 'file': '1484.npy'}","{'folder': 'segmentation', 'file': '1484.npy'}",True
1496,"{'folder': 'frame', 'file': '1496.npy'}","{'folder': 'segmentation', 'file': '1496.npy'}",True
1497,"{'folder': 'frame', 'file': '1497.npy'}","{'folder': 'segmentation', 'file': '1497.npy'}",True


# Benchmark Endoseg

In [24]:
import time
import numpy as np
import endoseg

from utils import IoU

# load model with default values https://github.com/luiscarlosgph/endoseg/blob/25dc910836193f8cc5ccccaddd78e95aada46682/src/run.py#L45
segmenter = endoseg.Segmenter(
    min_hsv_thresh=[0, 0, 0], 
    max_hsv_thresh=[255, 255, 10], 
    deinterlace=False, denoise=True
)

# log df
logs_endoseg = pd.DataFrame(columns=['time', 'iou'])

for idx, row in test_df.iterrows():
    # load and prepare data
    img = np.load(os.path.join(prefix, row.image['folder'], row.image['file']))
    seg_gt = np.load(os.path.join(prefix, row.segmentation['folder'], row.segmentation['file']))

    # benchmark execution time
    start = time.time()
    seg = segmenter.segment(img)
    execution_time = time.time() - start
    
    # benchmark iou
    iou = IoU(seg, seg_gt)

    # store logs
    logs_endoseg = logs_endoseg.append(
        {
            'time': execution_time,
            'iou': iou
        }, ignore_index=True
    )

# Benchmark Deepseg

In [25]:
import time
import numpy as np
from kornia import image_to_tensor, tensor_to_image
from lightning_modules import ImageSegmentationModule

from utils import IoU, load_yaml

device = 'cuda' # 'cpu' for cpu exection

# load model with default values
checkpoint_prefix = 'checkpoints/resnet_34'
checkpoint = 'epoch=35-step=1079.ckpt'

config = load_yaml(os.path.join(prefix, checkpoint_prefix, 'config.yml'))

model = ImageSegmentationModule(**config['model'])
model = model.load_from_checkpoint(os.path.join(prefix, checkpoint_prefix, checkpoint), **config['model'])
model = model.eval()
model.freeze()
model = model.to(device)

# log df
logs_deepseg = pd.DataFrame(columns=['time', 'iou'])

for idx, row in test_df.iterrows():
    # load and prepare data
    img = np.load(os.path.join(prefix, row.image['folder'], row.image['file']))
    seg_gt = np.load(os.path.join(prefix, row.segmentation['folder'], row.segmentation['file']))

    # benchmark execution time
    start = time.time()
    img = image_to_tensor(img, keepdim=False)
    img = img.to(model.device)
    img = img.float()/255.
    seg = model(img)
    execution_time = time.time() - start

    # benchmark iou
    seg = tensor_to_image(seg, keepdim=False)
    iou = IoU(seg, seg_gt)

    # store logs
    logs_deepseg = logs_deepseg.append(
        {
            'time': execution_time,
            'iou': iou
        }, ignore_index=True
    )

In [26]:
print('Endoseg: Execution time {:.4f} +/- {:.4f}, Rate {:.1f}, IoU: {:.3f} +/- {:.3f}'.format(
        logs_endoseg.time.mean(), 
        logs_endoseg.time.std(), 
        1/logs_endoseg.time.mean(), 
        logs_endoseg.iou.mean(), 
        logs_endoseg.iou.std()
    )
)

print('Deepseg: Execution time {:.4f} +/- {:.4f}, Rate {:.1f}, IoU: {:.3f} +/- {:.3f}'.format(
        logs_deepseg.time.mean(), 
        logs_deepseg.time.std(), 
        1/logs_deepseg.time.mean(), 
        logs_deepseg.iou.mean(), 
        logs_deepseg.iou.std()
    )
)

Endoseg: Execution time 0.0660 +/- 0.0087, Rate 15.1, IoU: 0.969 +/- 0.066
Deepseg: Execution time 0.0052 +/- 0.0055, Rate 193.5, IoU: 0.995 +/- 0.005


# Export Deepseg

In [27]:
import torch
from lightning_modules import ImageSegmentationModule

from utils import load_yaml

device = 'cuda' # 'cpu' for cpu exection

# load model with default values
checkpoint_prefix = 'checkpoints/resnet_34_tiny'
checkpoint = 'epoch=43-step=1319.ckpt'

config = load_yaml(os.path.join(prefix, checkpoint_prefix, 'config.yml'))

model = ImageSegmentationModule(**config['model'])
model = model.load_from_checkpoint(os.path.join(prefix, checkpoint_prefix, checkpoint), **config['model'])
model = model.to(device)

script = model.to_torchscript(method='trace', example_inputs=torch.rand([1,3,320,480]))
torch.jit.save(script, '{}.pt'.format(device))

/home/martin/miniconda3/envs/fov_benchmark/lib/python3.8/site-packages/kornia/geometry/transform/affwarp.py:574: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if size == input_size:


In [28]:
import time
import numpy as np
from kornia import image_to_tensor, tensor_to_image
from lightning_modules import ImageSegmentationModule

from utils import IoU, load_yaml

device = 'cuda' # 'cpu' for cpu exection

model = torch.jit.load('{}.pt'.format(device))

# log df
logs_deepseg = pd.DataFrame(columns=['time', 'iou'])

for idx, row in test_df.iterrows():
    # load and prepare data
    img = np.load(os.path.join(prefix, row.image['folder'], row.image['file']))
    seg_gt = np.load(os.path.join(prefix, row.segmentation['folder'], row.segmentation['file']))

    # benchmark execution time
    start = time.time()
    img = image_to_tensor(img, keepdim=False)
    img = img.to(device)
    img = img.float()/255.
    seg = model(img)
    execution_time = time.time() - start

    # benchmark iou
    seg = tensor_to_image(seg, keepdim=False)
    iou = IoU(seg, seg_gt)

    # store logs
    logs_deepseg = logs_deepseg.append(
        {
            'time': execution_time,
            'iou': iou
        }, ignore_index=True
    )

In [29]:
print('Endoseg: Execution time {:.4f} +/- {:.4f}, Rate {:.1f}, IoU: {:.3f} +/- {:.3f}'.format(
        logs_endoseg.time.mean(), 
        logs_endoseg.time.std(), 
        1/logs_endoseg.time.mean(), 
        logs_endoseg.iou.mean(), 
        logs_endoseg.iou.std()
    )
)

print('Deepseg: Execution time {:.4f} +/- {:.4f}, Rate {:.1f}, IoU: {:.3f} +/- {:.3f}'.format(
        logs_deepseg.time.mean(), 
        logs_deepseg.time.std(), 
        1/logs_deepseg.time.mean(), 
        logs_deepseg.iou.mean(), 
        logs_deepseg.iou.std()
    )
)

Endoseg: Execution time 0.0660 +/- 0.0087, Rate 15.1, IoU: 0.969 +/- 0.066
Deepseg: Execution time 0.0033 +/- 0.0049, Rate 299.4, IoU: 0.993 +/- 0.006
